# Exam

## Part 1: Question

Compare BERT and GPT-2: high-level description 

BERT uses self-attention and encoder blocks, while GPT-2 uses masked self-attention to prevent the information leak from the future tokens and decoder blocks in auto-regresssive fashion to output one token at a time.

## Part 2: Coding

Develop a model for predicting review rating.

**Multiclass classification into 5 classes**

Score: **F1 with macro averaging**

You are forbidden to use test dataset for any kind of training.

Remember proper training pipeline.

If you are not using default params in the models, you have to use some 
validation scheme to justify them.

Use random_state or seed params - your experiment must be reprodusible.

**1 baseline = 0.51**

**2 baseline = 0.53**

In [0]:
import pandas as pd
import numpy as np
import sklearn
import spacy

SEED=1337
np.random.seed(SEED)

In [2]:
!unzip exam_data.zip

Archive:  exam_data.zip
  inflating: test.csv                
  inflating: train.csv               


In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.shape

(48192, 3)

In [3]:
df_train.head()

,review,title,target
0,"The staff was very friendly, the breakfast ver...",Walker Gem,5
1,Excellent service - very approachable and prof...,Excellent Service,4
2,Really a top notch place to spend a day at the...,"Good location, warm and friendly staff",5
3,"a little noisy, there was a false fire alarm a...","nice hotel,",4
4,Place had too many animals and I'm allergic to...,Experience,3


In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
import numpy as np
from collections import Counter
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

In [0]:
vectorizer = TfidfVectorizer()
df_train['text'] = df_train.title + '. ' + df_train.review
X_tv = vectorizer.fit_transform(df_train.text)
df_test['text'] = df_test.title + '. ' + df_test.review
X_test = vectorizer.transform(df_test.text)

In [0]:
y_tv = df_train.target
y_test = df_test.target
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=0.2, random_state=SEED)

In [7]:
X_train.shape, X_val.shape

((38553, 43520), (9639, 43520))

In [0]:
scoring = ['f1_macro']

In [0]:
skf = StratifiedKFold(5, shuffle=True, random_state=SEED)

### Logistic regression

In [0]:
clf = LogisticRegression(solver='saga', random_state=SEED)
param_dist = dict(C=uniform(loc=0, scale=4), penalty=['l2', 'l1'])
tuned_clf = RandomizedSearchCV(clf, param_dist, random_state=SEED, scoring=scoring, refit=scoring[0], cv=skf, n_jobs=-1)
search = tuned_clf.fit(X_tv, y_tv)
clf = search.best_estimator_

In [12]:
pred = clf.predict(X_test)
print(Counter(pred))
sklearn.metrics.f1_score(y_test, pred, average='macro')

Counter({5: 2464, 4: 1490, 3: 697, 1: 437, 2: 267})


0.5263214361628027

### RF

In [13]:
clf = RandomForestClassifier(n_estimators=50, random_state=SEED, oob_score=True, n_jobs=-1)
clf.fit(X_tv, y_tv)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
                       oob_score=True, random_state=1337, verbose=0,
                       warm_start=False)

In [14]:
pred = clf.predict(X_test)
print(Counter(pred))
sklearn.metrics.f1_score(y_test, pred, average='macro')

Counter({5: 3356, 4: 1278, 3: 355, 1: 301, 2: 65})


0.3687567631703554

### SVM

In [11]:
clf = SVC(kernel='linear', random_state=SEED)
clf.fit(X_tv, y_tv)
pred = clf.predict(X_test)
sklearn.metrics.f1_score(y_test, pred, average='macro')

0.5239671328122044